## Use Case: Log Request

In [24]:
import pickle
from abc import ABC, abstractmethod

class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

    def store(self):
        return pickle.dumps(self)
    
    @staticmethod
    @abstractmethod
    def load(data):
        return pickle.loads(data)

class AddItemCommand(Command):
    def __init__(self, item, target_list=None):
        self.item = item
        self.target_list = target_list
    
    def execute(self):
        if self.target_list is not None:
            print(f"Adding item: {self.item}")  # Debug statement
            self.target_list.append(self.item)
    
    @staticmethod
    def load(data):
        return pickle.loads(data)

class RemoveItemCommand(Command):
    def __init__(self, item, target_list=None):
        self.item = item
        self.target_list = target_list
    
    def execute(self):
        if self.target_list is not None and self.item in self.target_list:
            print(f"Removing item: {self.item}")  # Debug statement
            self.target_list.remove(self.item)
    
    @staticmethod
    def load(data):
        return pickle.loads(data)


In [25]:
class CommandLog:
    def __init__(self, filename):
        self.filename = filename
    
    def store_command(self, command):
        with open(self.filename, 'ab') as file:
            file.write(command.store() + b'\n')
        print(f"Stored command: {command}")  # Debug statement
    
    def load_commands(self):
        commands = []
        try:
            with open(self.filename, 'rb') as file:
                while True:
                    data = file.readline()
                    if not data:
                        break
                    command = AddItemCommand.load(data)
                    commands.append(command)
                    print(f"Loaded command: {command}")  # Debug statement
        except FileNotFoundError:
            print("Log file not found.")
        return commands


In [26]:
def main():
    items = []
    command_log = CommandLog('commands.pkl')
    
    # Create and execute commands
    add_command = AddItemCommand('item1', items)
    add_command.execute()
    command_log.store_command(add_command)
    
    remove_command = RemoveItemCommand('item1', items)
    remove_command.execute()
    command_log.store_command(remove_command)
    
    # Simulate system crash and recovery
    items.clear()
    print(f"Items before recovery: {items}")
    
    # Load commands from log and re-execute them
    commands = command_log.load_commands()
    for command in commands:
        command.target_list = items  # Reassign the target list
        command.execute()
    
    print(f"Items after recovery: {items}")

if __name__ == "__main__":
    main()

Adding item: item1
Stored command: <__main__.AddItemCommand object at 0x0000024CB4AD7370>
Removing item: item1
Stored command: <__main__.RemoveItemCommand object at 0x0000024CB491AA30>
Items before recovery: []
Loaded command: <__main__.AddItemCommand object at 0x0000024CB4B0CC70>
Loaded command: <__main__.RemoveItemCommand object at 0x0000024CB4B0CF40>
Loaded command: <__main__.AddItemCommand object at 0x0000024CB4B0C1F0>
Loaded command: <__main__.RemoveItemCommand object at 0x0000024CB4AD7DC0>
Loaded command: <__main__.AddItemCommand object at 0x0000024CB4AD7C40>
Loaded command: <__main__.RemoveItemCommand object at 0x0000024CB4AD7F70>
Loaded command: <__main__.AddItemCommand object at 0x0000024CB4AD7FA0>
Loaded command: <__main__.RemoveItemCommand object at 0x0000024CB4AD7A00>
Loaded command: <__main__.AddItemCommand object at 0x0000024CB4AD7AF0>
Loaded command: <__main__.RemoveItemCommand object at 0x0000024CB4AD71C0>
Loaded command: <__main__.AddItemCommand object at 0x0000024CB4A